# Digital Lab 3 - Power Flow Calculations
Author: Emil G. Melfald, University of South-Eastern Norway <br><br>
This notebook is the start of the electric power analysis python series, focusing on techniques applicable to **Electric Power Analysis**. 

## Prerequisites
Before proceeding, students should thoroughly understand the concepts introduced in **DigiLab part 1 and 2**. Additionally, familiarity with the power flow formulation is adviced. 

## Learning Objectives
By the end of this notebook, you will be able to:
- **Formulate** the fower flow equations  
- **Solve** the power flow using **Scipy**.
- Use **PyPSA** to execute power flow calculations. 

## Contents
1. [Busbar definitions in power flow studies](#Busbars)
2. [Setting up the system of equations](#Equations)
3. [Solving the power flow problem](#SolvePF)
---

In [1]:
import numpy as np 
from scipy.optimize import root 

# [Busbar definitions in power flow studies](#Busbars)
You should know that we may represent power system topology through an admittance matrix, $Y_{bus}$. 

$$ Y = \begin{bmatrix} Y_{11} & Y_{12} & ... & Y_{1n} \\ Y_{21} & ... & ... & Y_{2n} \\ ... & ... & ... & ... \\ Y_{n1} & Y_{n2} & ... & Y_{nn} \end{bmatrix} $$

Additionally, you should know about the power flow equations: 

$$ \begin{bmatrix} \bar{I_1} \\ \bar{I_2} \\ ... \\ \bar{I_n} \end{bmatrix} = Y_{bus} \begin{bmatrix} \bar{V_1} \\ \bar{V_2} \\ ... \\ \bar{V_n} \end{bmatrix} $$

In power flow studies, we normally use power injection instead of current injection for convenience of specifying loads and injected power. We convert the injected current to injected power by multiplying with the voltages, as we know $\bar{S}=\bar{V}\cdot \bar{I^*}$.  

$$ \begin{bmatrix} \bar{S_1} \\ \bar{S_2} \\ ... \\ \bar{S_n} \end{bmatrix} = \begin{bmatrix} \bar{V_1} & 0 & ... & 0 \\ 0 & \bar{V_2} & ... & ... \\ ... & ... & ... & ... \\ 0 & 0 & ... & \bar{V_n} \end{bmatrix} \begin{bmatrix} \bar{I_1^*} \\ \bar{I_2^*} \\ ... \\ \bar{I_n^*} \end{bmatrix} $$


A rather more intuitive and useful definition of the power flow in the system comes from separating the active and reactive power. This leads to the following two equations: 

$$ P_i = V_i \sum_{k=1}^{N_{\text{bus}}} V_k (G_{ik} \cos \theta_{ik} + B_{ik} \sin \theta_{ik}) $$

$$ Q_i = V_i \sum_{k=1}^{N_{\text{bus}}} V_k (G_{ik} \sin \theta_{ik} - B_{ik} \cos \theta_{ik}) $$

- $P_i$ - Active power injection into busbar i (positive means generation, negative means demand). 
- $Q_i$ - Reactive power injection into busbar i (positive means production, negative means demand). 
- $V_i$ - Voltage magnitude at bus i. 
- $V_k$ - Voltage magnitude at bus k. 
- $\theta_i$ - Voltage angle at bus i. 
- $\theta_k$ - Voltage angle at bus k. 
- $N_{\text{bus}}$ - Number of busbars in the system. 
- $G_{ik}$ - Conductance between bus i and bus k. 
- $B_{ik}$ - Suceptance between bus i and bus k. 


# [Setting up the system of equations](#Equations)
The goal when solving the power flow equations is to find the variables $y$ that causes the difference between calculated power flow and specified power flow to be as small as possible. We often call this the **power flow error** (PFE) $\Delta P$ and $\Delta Q$. For all busbars that we know either the active power or the reactive power we can specify a power flow error equation. 

$$ 0 = \Delta P_i = P_i - P_i^{calc}(y) \forall i \in [\text{PV and PQ buses}] $$

$$ 0 = \Delta Q_i = Q_i - Q_i^{calc}(y) \forall i \in [\text{PQ buses}] $$

**For PQ buses: 2 PFEs, 2 unknown variables**
- P - known 
- Q - known 
- V - unknown 
- $\theta$ - unknown 

**For PV buses: 1 PFE, 1 unknown variable**
- P - known 
- Q - unknown 
- V - known 
- $\theta$ - unknown

**For Slack Buses 0 PFE, 0 unknown variable**
- P - unknown 
- Q - unknown 
- V - known 
- $\theta$ - known

## Example system: 
In the system described below the $Y_{bus}$ matrix is given in the supplementary file. <br> 

![Example system](example1_Powerflow.png)

### Identifying variables and equations: 
In this case, we have two unknowns $y = [\theta_2, V_2]$. This comes from that both variables are known at the slack bus, and no variables are known at the PQ bus. 

In [2]:
from Digital_lab_3_supplementary_file import Y_bus_1
S_base = 100 # MW 
V_base = 22 # kV

V_vector = np.array([1.0, 1.0]) # We know V at index 0. We initialize V at PQ bus at 1.0 pu 
theta_vector = np.array([0.0, 0.0]) # We know theta at index 0 and guess it at the PQ bus. 
P_vector = np.array([0.0, -1.5])/S_base # We know the PQ power. Negative for demand. Remember to convert to pu 
Q_vector = np.array([0.0, -0.5])/S_base 

def powerflow_equations(y): 
    theta1, V1 = y 
    V_vec = V_vector.copy() 
    th_vec = theta_vector.copy() 

    # Insert the variables into the correct vectors before calculating the power flow
    th_vec[1] = theta1 
    V_vec[1] = V1 

    # Calculating the active and reactive power injection for each busbar
    P_calc = [] 
    Q_calc = []
    for i in range(len(V_vec)): 
        P = 0 
        Q = 0
        for k in range(len(V_vec)): 
            G_ik = Y_bus_1[i,k].real
            B_ik = Y_bus_1[i,k].imag
            theta_ik = th_vec[i]-th_vec[k]
            P += V_vec[i]*V_vec[k]*(G_ik*np.cos(theta_ik) + B_ik*np.sin(theta_ik))
            Q += V_vec[i]*V_vec[k]*(G_ik*np.sin(theta_ik) - B_ik*np.cos(theta_ik))
            
        P_calc.append(P)
        Q_calc.append(Q) 
    
    P_error = P_vector[1] - P_calc[1]
    Q_error = Q_vector[1] - Q_calc[1]
    return np.array([P_error, Q_error])


# [Solving the power flow problem](#SolvePF)

In [3]:
# We start by trying an arbitrary guess for theta2 and V2: 
# y = [theta2, V2]
pf_error0 = powerflow_equations(np.array([0.0, 1.0]))
pf_error1 = powerflow_equations(np.array([0.0, 0.99]))
pf_error2 = powerflow_equations(np.array([-0.1, 0.99]))
print(f"pf_error0 = {pf_error0.round(5)}, pf_error1 = {pf_error1.round(5)}, pf_error2 = {pf_error2.round(5)}")

pf_error0 = [-0.015 -0.005], pf_error1 = [-0.01303  0.00025], pf_error2 = [ 0.03841 -0.02203]


In [4]:
# We use scipy.optimize.root to find a solution to the power flow equations
y0 = np.array([0.0, 1.0]) # Flat start - all angles = 0, all voltages = 1 as initial guess
sol = root(powerflow_equations, y0)

sol # Notice sol.fun is very small, and sol.x is our solution for theta2 and V2

 message: The solution converged.
 success: True
  status: 1
     fun: [-1.403e-14 -2.364e-14]
       x: [-2.210e-02  9.819e-01]
    nfev: 8
    fjac: [[-9.255e-01  3.788e-01]
           [-3.788e-01 -9.255e-01]]
       r: [ 5.556e-01 -2.845e-02  5.475e-01]
     qtf: [ 1.998e-12  1.308e-11]

In [5]:
# Inserting the solution to the voltage and angle vectors 
theta_sol_vector = theta_vector.copy() 
theta_sol_vector[1] = sol.x[0]
V_sol_vector = V_vector.copy() 
V_sol_vector[1] = sol.x[1]

# Calculate the complex power in each busbar
V_sol_complex = V_sol_vector*(np.cos(theta_sol_vector) + 1j*np.sin(theta_sol_vector)) # Create a complex vector array 
I_inj = Y_bus_1 @ V_sol_complex
S_inj = V_sol_complex * np.conj(I_inj) 
S_inj *= S_base

print(f"Injected active power: {S_inj.real.round(3)} MW")
print(f"Injected reactive power: {S_inj.imag.round(3)} Mvar")
print(f"Voltages: {V_sol_vector.round(3)} pu")
print(f"Voltage angles: {(theta_sol_vector*180/np.pi).round(3)} degrees")

Injected active power: [ 1.516 -1.5  ] MW
Injected reactive power: [ 0.543 -0.5  ] Mvar
Voltages: [1.    0.982] pu
Voltage angles: [ 0.    -1.266] degrees


# Using PyPSA (Python library)
PyPSA is a python library that can execute power flow calculatons on power systems. It has a friendly user interface and is well documented on their hompepage [PyPSA Homepage](https://pypsa.readthedocs.io/en/latest/introduction.html). Following is a showcase of how the calculations above may be done in PyPSA. 

In [6]:
import pypsa
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # Adding this to remove spamming of "FutureWarning" from PyPSA

network = pypsa.Network()

network.add("Bus", "Bus 1", v_nom=22.0)
network.add("Bus", "Bus 2", v_nom=22.0)

network.add("Line", "Line 1", bus0="Bus 1", bus1="Bus 2", r=3.0, x=8.0)
network.add("Generator", "G1", bus="Bus 1", control="Slack")
network.add("Load", "Load 1", bus="Bus 2", p_set=1.5, q_set=0.5)

In [7]:
info = network.pf()
info

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.011781 seconds


{'n_iter': SubNetwork  0
 snapshot     
 now         3,
 'error': SubNetwork             0
 snapshot                
 now         9.431345e-12,
 'converged': SubNetwork     0
 snapshot        
 now         True}

In [8]:
P_sol = network.buses_t.p.values[0]
Q_sol = network.buses_t.q.values[0]
V_sol = network.buses_t.v_mag_pu.values[0]
th_sol = network.buses_t.v_ang.values[0]

print(f"Injected active power: {P_sol.round(3)} MW")
print(f"Injected reactive power: {Q_sol.round(3)} Mvar")
print(f"Voltages: {V_sol.round(3)} pu")
print(f"Voltage angles: {(th_sol*180/np.pi).round(3)} degrees")

Injected active power: [ 1.516 -1.5  ] MW
Injected reactive power: [ 0.543 -0.5  ] Mvar
Voltages: [1.    0.982] pu
Voltage angles: [ 0.    -1.266] degrees


#### End of Notebook
---